# 创建你的第一个异构联邦学习任务

## 创建任务

登录 [AlphaMed 管理页面](http://81.70.132.120/)。新建一个“异构联邦”计算任务，并根据需要填写其它任务信息，然后点击“创建任务”。

## demo 任务描述

我们基于 MNIST 任务射击了一个新的任务，以提供一个简单的容易入门的异构联邦学习示例。在这个示例任务中，每一张图片都被从图片正中切分成了左右两个部分。任务中共有两个参与方，每一方各持有一半的图片数据。如果每个参与方仅根据自己手中持有的半张图片判断手写数字，无疑准确率会大大的降低。但是如果双方能够精诚合作，将各自手中的信息最终汇总起来，一起完成手写数字识别的任务，就可以像拥有完整数据时一样获得很高的准确率了。

为了模拟各自只拥有一半图片的场景，我们修改了 MNIST 的 dataset 对象，使其返回的图像数据在进入模型训练前，抹除左（右）半边的图像数据。同时，参与方的数据集在输入模型前，会丢弃当前数据对应的标签。这样参与方就无从得知当前数据的标签了，只能依赖与发起方合作，共同完成学习任务。

现在，可以定义我们的计算任务细节了。我们假定你已经熟悉了深度学习的相关知识，并且具备了利用 PyTorch 框架将模型概念转化为代码的能力。若非如此，建议你先去学习一下这些知识，然后再回来继续后面的旅程。

要定义一个异构联邦学习任务，你需要先明确参与任务的各方所承担的角色。在所有参与方中，必须有一方能够提供训练任务的标签，以定义模型最终的学习目标，这一方将作为发起方 (以下称 Host) 控制学习的流程。其他的参与方各自拥有与任务匹配的数据，将承担协作方 (以下称 Collaborator) 的角色。由于各方拥有的数据是异构的，且不同的参与方本身会面临不同的约束，从而使得联合建模可能无法实现。故而为了提高任务适配性，采用特征融合的方式配合进行学习。即各参与方分别构建自己的本地模型（以下称 feature_model），将本地数据经 feature_model 处理后获得特征表示。Host 需要根据任务需要再设计一个推理模型（以下称 infer_model）以获取任务推理结果。在训练时，Host 收集所有参与方的特征数据，传入 infer_model 获得输出、计算损失，然后通过反向梯度传播更新模型参数。

## 检查数据

任何一个学习任务都需要读入训练数据，所以我们的第一个任务是确认参与运算的节点是否能够成功的访问到训练数据。现实世界的数据千变万化，但是在输入模型之前，一定会通过特征工程方法转换为适合模型的数据结构。因此我们的关注点主要有两个：
- 是否能够成功访问到训练数据？</br></br>
在联邦学习环境中访问数据的方式与在中心化的环境中并没有多少不同。只是由于数据拥有者的多样性，需要考虑建立一个统一的规范，并且尽量提升加载数据代码的兼容性，以尽可能提高加载数据的成功率。  
我们推荐你在任务描述中明确指定数据的存储和访问方式，以帮助数据拥有者明确如何准备数据以及应该将它们存放在哪里。（**需要保证数据依然保存于本地的安全环境中，因此上传到一个开放的云存储环境并提供链接可不是一个好主意。**）

- 是否能够成功将训练数据转换为适合的数据结构？</br></br>
在联邦学习环境中，原始数据的格式可能存在一些差异，比如图片数据可能使用了不同的尺寸或者格式。为了尽量提高特征转换的正确率，代码应当尽量兼容更多的可能情况。
我们推荐你在任务描述中明确指定支持的数据格式，以帮助数据拥有者明确如何对数据做预处理。如果可能的话，也可以附带一些数据处理的指导。

我们提供了一个 DataChecker 基础类，你需要根据自己的任务需要实现其 verify\_data 接口，并在数据验证无误时返回 True，否则返回 False。我们同时还提供了一个 notebook 调试环境，你可以在这个环境中调试你的代码和数据。当你确认数据和代码均已准备就绪时，可以通过调用 DataChecker 的 execute\_verification 方法通知平台执行数据集验证。如果验证通过，本地的任务状态会进入下一个阶段。

In [1]:
import os
from typing import Tuple

import torchvision
from alphafed import logger
from alphafed.scheduler import DataChecker
from torch.utils.data import DataLoader


class DemoDataChecker(DataChecker):

    def verify_data(self) -> Tuple[bool, str]:
        """数据集的校验具体逻辑."""
        logger.info(f"Start dataset verification for task {self.task_id}.")
        root_dir = '/data/alphamed-federated/tutorials/'
        is_touch_succ, touch_err = self._touch_data(root_dir)
        if not is_touch_succ:
            return is_touch_succ, touch_err
        is_load_succ, load_err = self._load_data(root_dir)
        if not is_load_succ:
            return is_load_succ, load_err

        return True, 'Varification Complete.'

    def _touch_data(self, root_dir: str) -> Tuple[bool, str]:
        """检查需要的数据是否存在，是否能够访问到."""
        file_list = [
            't10k-images-idx3-ubyte',
            't10k-images-idx3-ubyte.gz',
            't10k-labels-idx1-ubyte',
            't10k-labels-idx1-ubyte.gz',
            'train-images-idx3-ubyte',
            'train-images-idx3-ubyte.gz',
            'train-labels-idx1-ubyte',
            'train-labels-idx1-ubyte.gz'
        ]
        full_paths = [os.path.join(root_dir, 'MNIST', 'raw', _file) for _file in file_list]
        for _file in full_paths:
            if not os.path.exists(_file) or not os.path.isfile(_file):
                return False, f'{_file} does not exist or is not a file.'

        return True, ''

    def _load_data(self, root_dir: str) -> Tuple[bool, str]:
        """检查需要的数据是否能够被成功的转化为输入张量和标签张量."""
        data_loader = DataLoader(
            torchvision.datasets.MNIST(
                root_dir,
                train=True,
                download=False,
                transform=torchvision.transforms.Compose([
                    torchvision.transforms.ToTensor(),
                    torchvision.transforms.Normalize((0.1307,), (0.3081,))
                ])
            )
        )
        if data_loader is not None and len(data_loader) > 0:
            return True, ''
        else:
            return False, f'Failed to load train data from directory {root_dir} .'


checker = DemoDataChecker(task_id='TASK_ID')
checker.execute_verification()

实际上，两步检查都不是必须的，以提供最大的灵活性。但是我们强烈建议你尽可能实现它们。否则可能导致运行时错误，届时调试的代价可能会高出很多。数据验证成功之后，数据集状态将变为“验证通过”。

## Host 定义任务细节

Host 需要定义一个 “HeteroNNHostScheduler” 的子类，并实现其中的一些方法。你可以在我们的 Notebook 编辑框中创建你的第一个异构联邦学习任务，就像这样：

In [4]:
import os
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple, Union

import cloudpickle as pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNHostScheduler


class DemoHeteroHost(HeteroNNHostScheduler):
    ...

HeteroNNHostScheduler 接收一些初始化参数，一些主要的参数说明如下：
- feature_key（必填参数）：每个参与方需要为自己指定一个唯一的 key，以帮助发起方区分来自不同参与方的特征数据；
- name（可选参数）：训练任务的名称，默认为任务 ID；
- max_rounds（必填参数）：最多训练轮次，达到此训练轮次后任务结束；
- calculation_timeout（可选参数）：计算超时时间，开始执行本地训练后，在此时间内没有提交参数更新结果将被视为超时；
- log_rounds（可选参数）：每隔几轮训练后，执行一次测试并记录当前模型测试结果；

接下来，让我们一步一步完成它。

### 定义和划分样本 ID

第一个任务是实现 “load\_local\_ids” 方法，它将返回本地数据样本的 ID 列表。在异构联邦任务场景下，不同参与方拥有不同的样本集合，需要先划定出大家共同拥有的样本交集，才可以在此集合基础上继续训练。否则会大量出现样本数据残缺不全的情况，影响模型的训练效果。

这里有两点需要注意：
- 这里的样本 ID 只是用来为每一个具体样本提供一个唯一的标识，同时保证同一个样本在不同的参与方拥有同样的标识，并不需要与具体业务中的 “ID” 概念有必然的联系。比如手机号、身份证号码、QQ 号等均可以作为 ID 使用，但要确保各参与方之间一致。
- 在样本求交集的过程中，已经使用了安全手段确保 ID 信息不会外泄。但是如果您有更高的安全需求，可以考虑使用敏感信息（比如手机号、身份证号码、QQ 号）的哈希值作为 ID，从而获得更高的安全性。

简单起见，在我们的 demo 示例中，取训练集中前 20000 个样本的ID、测试集中前 5000 个样本的 ID 加 100000 作为整个数据集的 ID。

In [5]:
def load_local_ids(self) -> List[str]:
    train_ids = [str(i) for i in range(0, 20000)]
    test_ids = [str(i) for i in range(100000, 105000)]
    return train_ids + test_ids

在得到所有参与方样本 ID 的交集之后，需要将样本交集划分为训练集和测试集两个部分。**划分的逻辑需要各参与方事先协商，以保证所有参与方的划分结果是一致的。** 否则在训练的采样过程中，会出现样本数据缺失的情况。

在我们的示例中，我们将样本放回其在 MNIST 数据集中原本的位置。我们可以通过 “split\_dataset” 来达到这个目的。

In [6]:
def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
    ids = [int(_id) for _id in id_intersection]
    ids.sort()
    train_ids = ids[:10000]
    test_ids = ids[10000:]

    logger.info(f'Got {len(train_ids)} intersecting samples for training.')
    logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

    return set(train_ids), set(test_ids)

### 定义 feature_model，处理本地数据

接下来要考虑的是如何定义自己的 feature\_model，将本地数据的特征抽取出来，以传递给上层的 infer\_model。需要实现 “make\_feature\_model” 方法，它将返回本地使用的 feature\_model 对象。幸运的是，这就是个普通的 torch.nn.Module 对象。

In [7]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 3))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=80, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=20)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 80)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return self.fc2(x)

In [8]:
def build_feature_model(self) -> nn.Module:
    return ConvNet()

然后，我们来为 feature\_model 搭配一个优化器，帮助我们自动处理梯度优化。

In [9]:
def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
    return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.5)

平台会自动将学习任务使用的 feature\_model 对象传入 “make\_feature\_optimizer” 方法中，以方便建立绑定关系。

### 将样本数据转化为特征

第二个任务是加载样本数据并将其转化为特征，需要实现 “iterate\_train\_feature” 和 “iterate\_test\_feature” 两个方法。它们是两个迭代器，每次分别返回一个批次的训练数据和测试数据对应的特征张量。这两个方法返回的都是普通的 torch.Tensor 对象。如前所述，我们在将数据送入模型之前，需要抹除图像的右半部分。

In [10]:
def _erase_right(self, _image: torch.Tensor) -> torch.Tensor:
    return _image[:, :, :, :14]

def iterate_train_feature(self,
                          feature_model: nn.Module,
                          train_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
    train_dataset = torchvision.datasets.MNIST(
        self.data_dir,
        train=True,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ])
    )
    train_ids: List = list(train_ids)
    train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

    train_dataset = Subset(train_dataset, train_ids)
    self.train_loader = DataLoader(train_dataset,
                                    batch_size=self.batch_size,
                                    shuffle=False)

    for _data, _labels in self.train_loader:
        _data = self._erase_right(_data)
        yield feature_model(_data), _labels

def iterate_test_feature(self,
                         feature_model: nn.Module,
                         test_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
    test_dataset = torchvision.datasets.MNIST(
        self.data_dir,
        train=False,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ])
    )
    test_ids = [(i - 100000) for i in test_ids]
    test_dataset = Subset(test_dataset, test_ids)
    self.test_loader = DataLoader(test_dataset,
                                    batch_size=self.batch_size,
                                    shuffle=False)

    for _data, _labels in self.test_loader:
        _data = self._erase_right(_data)
        yield feature_model(_data), _labels

平台会自动将学习任务使用的 feature\_model 对象和划分好的样本 ID 集合 train\_ids、test\_ids 传入 “iterate\_train\_feature”、“iterate\_test\_feature” 方法中，以方便建立绑定关系。

这里需要注意的是，**参与任务的各方要事先商议好采样的逻辑，以确保各参与方每次采样的样本列表是一致的、且顺序是对齐的，** 否则模型学习的将是错误的特征数据。在我们的示例中，我们使用 MNIST 自带的顺序采样逻辑（shuffle=False），因此可以保证满足上述要求。

### 定义 infer_model

对于 Host 来讲，收集到所有参与方提交的特征信息后，还需要将它们融合后送入 infer\_model 以进一步处理，获得最终的输出。

In [ ]:
class InferModule(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(40, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, input):
        out = F.relu(self.fc1(input))
        out = self.fc2(out)
        return F.log_softmax(out, dim=-1)

In [ ]:
def build_infer_model(self) -> nn.Module:
    return InferModule()

然后，我们来为 infer\_model 搭配一个优化器，帮助我们自动处理梯度优化。

In [ ]:
def build_infer_optimizer(self, infer_model: nn.Module) -> optim.Optimizer:
    return optim.SGD(infer_model.parameters(), lr=0.01, momentum=0.5)

平台会自动将学习任务使用的 infer\_model 对象传入 “make\_infer\_optimizer” 方法中，以方便建立绑定关系。

### 定义训练逻辑

现在，可以定义我们的训练过程了。

在异构联邦学习结构中，核心的训练流程逻辑是在 infer\_model 中进行控制的。由于 infer\_model 是以融合后的特征张量作为输入的，因此 Host 在收集到所有参与方的特征信息后，需要先将它们融合为 infer\_model 需要的形式。融合的具体方式可以根据实际任务自由选择，既可以简单的拼接、池化，也可以分别送入不同的子网络模块。但是请一定要注意，**如果融合的过程中使用了可训练的模型和参数，则必须自己处理它们的训练、保存和加载，平台不会跟踪这里自定义的任何模型和参数。** 在我们的示例中，由于两个参与方分别获得了左右一半图像的特征，因此我们将它们拼接起来，以获得完整的图像特征。

In [ ]:
def train_a_batch(self, feature_projection: Dict[str, torch.Tensor], labels: torch.Tensor):
    fusion_tensor = torch.concat((feature_projection['demo_host'],
                                  feature_projection['demo_collaborator']), dim=1)
    self.optimizer.zero_grad()
    out = self.infer_model(fusion_tensor)
    loss = F.nll_loss(out, labels)
    loss.backward()
    self.optimizer.step()

平台会自动将每一个批次中各参与方提交的特征信息和样本对应的标签传入 “train\_a\_batch” 方法中，在降特征传入 infer\_model 前，需要先将它们融合。后续求损失和梯度的操作与标准的模型训练过程无异。

为方便使用，平台提供了 “self.optimizer.zero\_grad()” 和 “self.optimizer.step()”，分别处理梯度清理和更新。它们会统一处理 feature\_model 和 infer\_model 的相关工作，如果没有特殊的需要，建议直接使用。**（这两个工具方法在简化版的异构联邦学习中看起来有些鸡肋，那是因为它们其实是针对加密环境下的异构联邦计算设计的。在加密环境下，特征融合和梯度的处理过程要复杂的多，已经超出了普通的 pytorch.optim 优化机制的能力范畴。此时这两个方法将成为必须使用的工具。）**

### 定义测试逻辑

为了验证模型的训练成果，我们需要在必要的时候对当前的最新参数做一些测试。同样的，测试的方式与本地测试也是一模一样的。如果需要在训练的过程中记录一些评估指标，以便训练完成之后做一些分析，可以通过 TensorBoard 记录日志数据。但是需要注意的是，处于安全的原因，平台对文件系统的访问是有限制的，因此不能为 TensorBoard 日志任意指定保存目录。平台为此提供了 `tb_writer` 工具，其不仅可以记录指标数据，还能够支持在训练完成时导出 TensorBoard 日志。相反，**如果随意创建 Writer 对象记录 TensorBoard 日志，可能导致记录失败或无法访问到记录的数据。**

**`register_metrics` 机制已经停止维护并被移除，请尽快升级**

In [ ]:
def test(self,
         batched_feature_projections: List[torch.Tensor],
         batched_labels: List[torch.Tensor]):
    start = time()
    test_loss = 0
    correct = 0
    for _feature_projection, _lables in zip(batched_feature_projections, batched_labels):
        fusion_tensor = torch.concat((_feature_projection['demo_host'],
                                      _feature_projection['demo_collaborator']), dim=1)
        out: torch.Tensor = self.infer_model(fusion_tensor)
        test_loss += F.nll_loss(out, _lables)
        pred = out.max(1, keepdim=True)[1]
        correct += pred.eq(_lables.view_as(pred)).sum().item()

    test_loss /= len(self.test_loader.dataset)
    accuracy = correct / len(self.test_loader.dataset)
    correct_rate = 100. * accuracy
    logger.info(f'Test set: Average loss: {test_loss:.4f}')
    logger.info(
        f'Test set: Accuracy: {accuracy} ({correct_rate:.2f}%)'
    )

    end = time()

    self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
    self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
    self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
    self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

好了，到此为止，所有必须的工作都已经做完了。下面的步骤是可选的，你可以根据自己的需要选择实现。当然，也可以全部跳过。

### 定义可选的任务细节

#### 添加自己的初始化配置

在大多数现实场景中，默认的初始化配置项可能都不足以满足你的全部需求。依据你使用的模型和训练方法，你可能需要加入更多的参数以获得更好的训练效果。要实现这一点很简单，就像所有普通的 python 类定义一样，你只需要在 “\_\_init__” 方法中做一些必要的处理。比如在我们的 demo 中，我们就在 “\_\_init__” 方法中初始化了训练集和测试集的 DataLoader。这里还需要注意的是，如果定义了类似 “batch\_size” 这样的采样参数，需要确保各参与方每次采样的样本数量一致。

In [ ]:
def __init__(self,
             feature_key: str,
             batch_size: int,
             data_dir: str,
             max_rounds: int = 0,
             calculation_timeout: int = 300,
             schedule_timeout: int = 30,
             log_rounds: int = 0) -> None:
    super().__init__(feature_key=feature_key,
                     max_rounds=max_rounds,
                     calculation_timeout=calculation_timeout,
                     schedule_timeout=schedule_timeout,
                     log_rounds=log_rounds)
    self.batch_size = batch_size
    self.data_dir = data_dir

在现实场景中，我们可能需要添加多个有助于模型训练的参数。也许其中有一些是需要在外部控制的，比如 “batch\_size”、“learning\_rate”、“momentum”，你可以把它们加入初始化的参数列表中。也许还有一些参数不希望受到外部环境的影响，比如 “device”、“seed”，那就把它们藏在 “\_\_init__” 内部好了。这样可以避免外部使用者错误的设置这些参数，同时还能保证训练过程中随时可以访问到这些参数。

**请记住，所有这些附加的参数，都需要你自己来管理。**

#### 验证运行环境

在实际运行之前，可能会希望对运行环境再做一些检查，帮助发现一些潜在的错误。如果你确实有此需求，可以实现 “validate\_context” 方法，添加任何你需要的检查逻辑，或者输出一些环境信息以利于检查问题。但是需要留意，**不要忘记先调用父类的方法，否则会导致运行时错误。**

如果你没有这方面的需求，则可以直接跳过这一步，使用平台的默认实现。

In [ ]:
def validate_context(self):
    # do something
    ...

#### 控制任务完成的条件

默认情况下，计算任务将在完成 max\_rounds 轮的训练之后自动完成。在一些更复杂的场景中，你可能希望使用一些更复杂的逻辑以判断是否要结束训练，甚至可能希望训练永远执行下去。此时就需要修改 “is\_task\_finished” 方法的判断逻辑了，将它修改成你希望的样子吧。

In [ ]:
def is_task_finished(self) -> bool:
    """By default true if reach the max rounds configured."""
    return self._is_reach_max_rounds()

### 启动 Host 任务

至此，一个可以运行的 Host 已经基本准备就绪了。在真正启动我们的第一个任务之前，让我们先把前面那些零散的方法实现整理一下，汇总到一起。然后，你还需要在任务管理页面中查看一下当前任务的 ID。任务 ID 可以在 Playgroud 页面找到并复制，如下图：

![获取当前任务 ID](../resource/task_id.png)

现在，可以启动 Host 程序了。等到所有的参与方全部上线之后，它会自动启动训练流程。由于异构联邦学习任务中需要大量的沟通交互，因此学习速度会显著慢于本地训练流程，因此我们需要有一些耐心。这个 demo 完成训练大概会需要两个小时左右。

In [ ]:
import os
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple, Union

import cloudpickle as pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNHostScheduler

class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 3))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=80, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=20)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 80)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return self.fc2(x)


class InferModule(nn.Module):

    def __init__(self) -> None:
        super().__init__()
        self.fc1 = nn.Linear(40, 20)
        self.fc2 = nn.Linear(20, 10)

    def forward(self, input):
        out = F.relu(self.fc1(input))
        out = self.fc2(out)
        return F.log_softmax(out, dim=-1)


class DemoHeteroHost(HeteroNNHostScheduler):

    def __init__(self,
                 feature_key: str,
                 batch_size: int,
                 data_dir: str,
                 max_rounds: int = 0,
                 calculation_timeout: int = 300,
                 schedule_timeout: int = 30,
                 log_rounds: int = 0) -> None:
        super().__init__(feature_key=feature_key,
                         max_rounds=max_rounds,
                         calculation_timeout=calculation_timeout,
                         schedule_timeout=schedule_timeout,
                         log_rounds=log_rounds)
        self.batch_size = batch_size
        self.data_dir = data_dir

    def load_local_ids(self) -> List[str]:
        train_ids = [str(i) for i in range(0, 20000)]
        test_ids = [str(i) for i in range(100000, 105000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        return ConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_right(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, :14]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        train_dataset = torchvision.datasets.MNIST(
            self.data_dir,
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _labels in self.train_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> Tuple[torch.Tensor, torch.Tensor]:
        test_dataset = torchvision.datasets.MNIST(
            self.data_dir,
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _labels in self.test_loader:
            _data = self._erase_right(_data)
            yield feature_model(_data), _labels

    def build_infer_model(self) -> nn.Module:
        return InferModule()

    def build_infer_optimizer(self, infer_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(infer_model.parameters(), lr=0.01, momentum=0.9)

    def train_a_batch(self, feature_projection: Dict[str, torch.Tensor], labels: torch.Tensor):
        fusion_tensor = torch.concat((feature_projection['demo_host'],
                                      feature_projection['demo_collaborator']), dim=1)
        self.optimizer.zero_grad()
        out = self.infer_model(fusion_tensor)
        loss = F.nll_loss(out, labels)
        loss.backward()
        self.optimizer.step()

    def test(self,
             batched_feature_projections: List[torch.Tensor],
             batched_labels: List[torch.Tensor]):
        start = time()
        test_loss = 0
        correct = 0
        for _feature_projection, _lables in zip(batched_feature_projections, batched_labels):
            fusion_tensor = torch.concat((_feature_projection['demo_host'],
                                          _feature_projection['demo_collaborator']), dim=1)
            out: torch.Tensor = self.infer_model(fusion_tensor)
            test_loss += F.nll_loss(out, _lables)
            pred = out.max(1, keepdim=True)[1]
            correct += pred.eq(_lables.view_as(pred)).sum().item()

        test_loss /= len(self.test_loader.dataset)
        accuracy = correct / len(self.test_loader.dataset)
        correct_rate = 100. * accuracy
        logger.info(f'Test set: Average loss: {test_loss:.4f}')
        logger.info(
            f'Test set: Accuracy: {accuracy} ({correct_rate:.2f}%)'
        )

        end = time()

        self.tb_writer.add_scalar('timer/run_time', end - start, self.current_round)
        self.tb_writer.add_scalar('test_results/average_loss', test_loss, self.current_round)
        self.tb_writer.add_scalar('test_results/accuracy', accuracy, self.current_round)
        self.tb_writer.add_scalar('test_results/correct_rate', correct_rate, self.current_round)

scheduler = DemoHeteroHost(feature_key='demo_host',
                           batch_size=128,
                           data_dir='/data/alphamed-federated/tutorials/',
                           max_rounds=1,
                           calculation_timeout=300,
                           log_rounds=1)
scheduler.launch_task(task_id='YOUR_TASK_ID')

## Collaborator 定义任务细节

Collaborator 需要定义一个 “HeteroNNCollaboratorScheduler” 的子类，并实现其中的一些方法。你可以在我们的 Notebook 编辑框中创建你的第一个异构联邦学习任务，就像这样：

In [ ]:
import os
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple, Union

import cloudpickle as pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNCollaboratorScheduler


class DemoHeteroCollaborator(HeteroNNCollaboratorScheduler):
    ...

HeteroNNCollaboratorScheduler 接收一些初始化参数，一些主要的参数说明如下：
- feature_key（必填参数）：每个参与方需要为自己指定一个唯一的 key，以帮助发起方区分来自不同参与方的特征数据；
- name（可选参数）：训练任务的名称，默认为任务 ID；

接下来，让我们一步一步完成它。

### 定义和划分样本 ID

第一个任务是实现 “load\_local\_ids” 方法，它将返回本地数据样本的 ID 列表。在异构联邦任务场景下，不同参与方拥有不同的样本集合，需要先划定出大家共同拥有的样本交集，才可以在此集合基础上继续训练。否则会大量出现样本数据残缺不全的情况，影响模型的训练效果。

这里有两点需要注意：
- 这里的样本 ID 只是用来为每一个具体样本提供一个唯一的标识，同时保证同一个样本在不同的参与方拥有同样的标识，并不需要与具体业务中的 “ID” 概念有必然的联系。比如手机号、身份证号码、QQ 号等均可以作为 ID 使用，但要确保各参与方之间一致。
- 在样本求交集的过程中，已经使用了安全手段确保 ID 信息不会外泄。但是如果您有更高的安全需求，可以考虑使用敏感信息（比如手机号、身份证号码、QQ 号）的哈希值作为 ID，从而获得更高的安全性。

简单起见，在我们的 demo 示例中，简单的使用训练集中第 10000 - 30000 个样本的ID、测试集中第 3000 - 7000 个样本的 ID。最终求交后会包含 10000 个训练样本和 2000 个测试样本。

In [ ]:
def load_local_ids(self) -> List[str]:
    train_ids = [str(i) for i in range(10000, 30000)]
    test_ids = [str(i) for i in range(103000, 107000)]
    return train_ids + test_ids

在得到所有参与方样本 ID 的交集之后，需要将样本交集划分为训练集和测试集两个部分。**划分的逻辑需要与发起方事先协商，以保证所有参与方的划分结果是一致的。** 否则在训练的采样过程中，会出现样本数据缺失的情况。

在我们的示例中，我们将样本放回其在 MNIST 数据集中原本的位置。我们可以通过 “split\_dataset” 来达到这个目的。

In [ ]:
def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
    ids = [int(_id) for _id in id_intersection]
    ids.sort()
    train_ids = ids[:10000]
    test_ids = ids[10000:]

    logger.info(f'Got {len(train_ids)} intersecting samples for training.')
    logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

    return set(train_ids), set(test_ids)

### 定义 feature_model，处理本地数据

接下来要考虑的是如何定义自己的 feature\_model，将本地数据的特征抽取出来，以传递给上层的 infer\_model。需要实现 “make\_feature\_model” 方法，它将返回本地使用的 feature\_model 对象。幸运的是，这就是个普通的 torch.nn.Module 对象。

In [ ]:
class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 3))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=80, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=20)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 80)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return self.fc2(x)

In [ ]:
def build_feature_model(self) -> nn.Module:
    return ConvNet()

然后，我们来为 feature\_model 搭配一个优化器，帮助我们自动处理梯度优化。

In [ ]:
def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
    return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.5)

平台会自动将学习任务使用的 feature\_model 对象传入 “make\_feature\_optimizer” 方法中，以方便建立绑定关系。

### 将样本数据转化为特征

第二个任务是加载样本数据并将其转化为特征，需要实现 “iterate\_train\_feature” 和 “iterate\_test\_feature” 两个方法。它们是两个迭代器，每次分别返回一个批次训练数据的特征张量。这两个方法返回的都是普通的 torch.Tensor 对象。如前所述，在将样本数据送入模型之前，我们需要抹除图像的左边部分，并且丢弃类别标签。

In [ ]:
def iterate_train_feature(self,
                          feature_model: nn.Module,
                          train_ids: Set[str]) -> torch.Tensor:
    train_dataset = torchvision.datasets.MNIST(
        self.data_dir,
        train=True,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ])
    )
    train_ids: List = list(train_ids)
    train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

    train_dataset = Subset(train_dataset, train_ids)
    self.train_loader = DataLoader(train_dataset,
                                    batch_size=self.batch_size,
                                    shuffle=False)

    for _data, _ in self.train_loader:
        _data = self._erase_left(_data)
        yield feature_model(_data)

def iterate_test_feature(self,
                         feature_model: nn.Module,
                         test_ids: Set[str]) -> torch.Tensor:
    test_dataset = torchvision.datasets.MNIST(
        self.data_dir,
        train=False,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize((0.1307,), (0.3081,))
        ])
    )
    test_ids = [(i - 100000) for i in test_ids]
    test_dataset = Subset(test_dataset, test_ids)
    self.test_loader = DataLoader(test_dataset,
                                    batch_size=self.batch_size,
                                    shuffle=False)

    for _data, _ in self.test_loader:
        _data = self._erase_left(_data)
        yield feature_model(_data)

平台会自动将学习任务使用的 feature\_model 对象和划分好的样本 ID 集合 train\_ids、test\_ids 传入 “iterate\_train\_feature”、“iterate\_test\_feature” 方法中，以方便建立绑定关系。

这里需要注意的是，**要事先与发起方商议好采样的逻辑，以确保各参与方每次采样的样本列表是一致的、且顺序是对齐的，** 否则模型学习的将是错误的特征数据。在我们的示例中，我们使用 MNIST 自带的顺序采样逻辑（shuffle=False），因此可以保证满足上述要求。

### 定义可选的任务细节

#### 添加自己的初始化配置

在大多数现实场景中，默认的初始化配置项可能都不足以满足你的全部需求。依据你使用的模型和训练方法，你可能需要加入更多的参数以获得更好的训练效果。要实现这一点很简单，就像所有普通的 python 类定义一样，你只需要在 “\_\_init__” 方法中做一些必要的处理。比如在我们的 demo 中，我们就在 “\_\_init__” 方法中初始化了训练集和测试集的 DataLoader。这里还需要注意的是，如果定义了类似 “batch\_size” 这样的采样参数，需要与发起方事先商议，确保各参与方每次采样的样本数量一致。

In [ ]:
def __init__(self,
             feature_key: str,
             batch_size: int,
             data_dir: str,
             schedule_timeout: int = 30,
             is_feature_trainable: bool = True) -> None:
    super().__init__(feature_key=feature_key,
                     schedule_timeout=schedule_timeout,
                     is_feature_trainable=is_feature_trainable)
    self.batch_size = batch_size
    self.data_dir = data_dir

在现实场景中，我们可能需要添加多个有助于模型训练的参数。也许其中有一些是需要在外部控制的，比如 “batch\_size”、“learning\_rate”、“momentum”，你可以把它们加入初始化的参数列表中。也许还有一些参数不希望受到外部环境的影响，比如 “device”、“seed”，那就把它们藏在 “\_\_init__” 内部好了。这样可以避免外部使用者错误的设置这些参数，同时还能保证训练过程中随时可以访问到这些参数。

**请记住，所有这些附加的参数，都需要你自己来管理。**

#### 验证运行环境

在实际运行之前，可能会希望对运行环境再做一些检查，帮助发现一些潜在的错误。如果你确实有此需求，可以实现 “validate\_context” 方法，添加任何你需要的检查逻辑，或者输出一些环境信息以利于检查问题。但是需要留意，**不要忘记先调用父类的方法，否则会导致运行时错误。**

如果你没有这方面的需求，则可以直接跳过这一步，使用平台的默认实现。

In [ ]:
def validate_context(self):
    # do something
    ...

### 启动 Collaborator 任务

至此，一个可以运行的 Collaborator 已经基本准备就绪了。在真正启动我们的第一个任务之前，让我们先把前面那些零散的方法实现整理一下，汇总到一起。然后，你还需要在任务管理页面中查看一下当前任务的 ID。任务 ID 可以在 Playgroud 页面找到并复制，如下图：

![获取当前任务 ID](../resource/task_id.png)

现在，可以启动 Collaborator 程序了。等到所有的参与方全部上线之后，Host 会自动启动训练流程。由于异构联邦学习任务中需要大量的沟通交互，因此学习速度会显著慢于本地训练流程，因此我们需要有一些耐心。这个 demo 完成训练大概会需要两个小时左右。

In [ ]:
import os
from hashlib import md5
from time import time
from typing import Dict, List, Set, Tuple, Union

import cloudpickle as pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader, Subset

from alphafed import logger
from alphafed.hetero_nn import HeteroNNCollaboratorScheduler

class ConvNet(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(5, 3))
        self.conv2 = nn.Conv2d(in_channels=10, out_channels=20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(in_features=80, out_features=50)
        self.fc2 = nn.Linear(in_features=50, out_features=20)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 80)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        return self.fc2(x)

class DemoHeteroCollaborator(HeteroNNCollaboratorScheduler):

    def __init__(self,
                 feature_key: str,
                 batch_size: int,
                 data_dir: str,
                 schedule_timeout: int = 30,
                 is_feature_trainable: bool = True) -> None:
        super().__init__(feature_key=feature_key,
                         schedule_timeout=schedule_timeout,
                         is_feature_trainable=is_feature_trainable)
        self.batch_size = batch_size
        self.data_dir = data_dir

    def load_local_ids(self) -> List[str]:
        train_ids = [str(i) for i in range(10000, 30000)]
        test_ids = [str(i) for i in range(103000, 107000)]
        return train_ids + test_ids

    def split_dataset(self, id_intersection: Set[str]) -> Tuple[Set[str], Set[str]]:
        ids = [int(_id) for _id in id_intersection]
        ids.sort()
        train_ids = ids[:10000]
        test_ids = ids[10000:]

        logger.info(f'Got {len(train_ids)} intersecting samples for training.')
        logger.info(f'Got {len(test_ids)} intersecting samples for testing.')

        return set(train_ids), set(test_ids)

    def build_feature_model(self) -> nn.Module:
        return ConvNet()

    def build_feature_optimizer(self, feature_model: nn.Module) -> optim.Optimizer:
        return optim.SGD(feature_model.parameters(), lr=0.01, momentum=0.9)

    def _erase_left(self, _image: torch.Tensor) -> torch.Tensor:
        return _image[:, :, :, 14:]

    def iterate_train_feature(self,
                              feature_model: nn.Module,
                              train_ids: Set[str]) -> torch.Tensor:
        train_dataset = torchvision.datasets.MNIST(
            self.data_dir,
            train=True,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        train_ids: List = list(train_ids)
        train_ids.sort(key=lambda x: md5(bytes(x + self.current_round)).digest())

        train_dataset = Subset(train_dataset, train_ids)
        self.train_loader = DataLoader(train_dataset,
                                       batch_size=self.batch_size,
                                       shuffle=False)

        for _data, _ in self.train_loader:
            _data = self._erase_left(_data)
            yield feature_model(_data)

    def iterate_test_feature(self,
                             feature_model: nn.Module,
                             test_ids: Set[str]) -> torch.Tensor:
        test_dataset = torchvision.datasets.MNIST(
            self.data_dir,
            train=False,
            download=True,
            transform=torchvision.transforms.Compose([
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])
        )
        test_ids = [(i - 100000) for i in test_ids]
        test_dataset = Subset(test_dataset, test_ids)
        self.test_loader = DataLoader(test_dataset,
                                      batch_size=self.batch_size,
                                      shuffle=False)

        for _data, _ in self.test_loader:
            _data = self._erase_left(_data)
            yield feature_model(_data)

scheduler = DemoHeteroCollaborator(feature_key='demo_collaborator',
                                   batch_size=128,
                                   data_dir='/data/alphamed-federated/tutorials/')
scheduler.launch_task(task_id='YOUR_TASK_ID')